In [1]:
import scipy.io
import numpy as np
from numpy import linalg as LA
import matplotlib
import time
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from functions import *
from numpy import random

In [2]:
# load data
mat = scipy.io.loadmat('face.mat')
raw_data = mat['X']

D,N = raw_data.shape

raw_data = np.transpose(raw_data)

partitioned_training_data = np.empty([4,int(520*0.8), 2576])

testing_data = np.empty([int(520*0.2), 2576])

# create training and test data
for x in range(52):
    for y in range(4):
        partitioned_training_data[y][x*2:(x+1)*2] = raw_data[x*10+(2*y):x*10+(2*(y+1))]
    testing_data[x*2:(x+1)*2] = raw_data[x*10+8:(x+1)*10]
    

raw_data = np.transpose(raw_data)
training_data = np.transpose(partitioned_training_data)
testing_data = np.transpose(testing_data)

mean = np.empty([4,2576])

for i in range(4):
    mean[i] = partitioned_training_data[i].mean()

Compute covariance matrix and mean of dataset

In [3]:
S = np.empty([4,416,416])
D = np.empty([4])
N = np.empty([4])
A = np.empty([4,416,2576])

for i in range(4):
    A[i] = partitioned_training_data[i] - mean[i]
    D[i],N[i] = A[i].shape
    S[i] = (1/N[i])*np.dot(A[i],A[i].T) 


Using only 1 partition of the training dataset

Combine partition 0 and 1

In [4]:
# new_mu = (N[0]*mean[0] + N[1]*mean[1])/(N[0]+N[1])
# new_cov = (N[0]/(N[0]+N[1]))*S[0] + (N[1]/(N[0]+N[1]))*S[1] + ((N[0]*N[1])/(N[0]+N[1])**2)*np.dot(mean[0]-mean[1],(mean[0]-mean[1]).T)
new_mu = combined_mean(N,mean,0,1)
new_cov = combined_cov(N,S,mean,0,1)

q,r = LA.qr(new_cov)

p = np.matmul(q,r)
w,v = LA.eig(new_cov)

u = np.matmul(A[0].T,v)

u /= LA.norm(u,ord=2,axis=0)
id = np.argsort(np.abs(w))[::-1]
w = w[id]
u = u[:,id].real
show_img(u[:,0])



new cov shape:  (416, 416)


In [5]:
input = testing_data
print(testing_data.shape)
mean_face = new_mu.reshape(-1,1)
eigenface = u
n = 100

(2576, 104)


In [6]:
faces = reconstruct(input, mean_face, eigenface, n)

In [7]:
print(faces)

[[192.82026403 190.86398007 185.18327846 ... 145.52666503 140.66404053
  175.85113315]
 [170.00844808 163.68638554 157.97457332 ... 154.18868991 154.42767895
  189.04272177]
 [217.01745086 220.85000927 237.4060511  ...  28.33020096   2.85684117
    9.76840287]
 ...
 [152.2064263  142.92543301 143.87142138 ... 289.34643327 267.7691117
  281.26726631]
 [199.52182393 202.34543541 215.77551548 ... 162.89768527 142.81498088
  166.4975354 ]
 [185.3388485  178.35362738 169.07434555 ... 168.39147289 134.1242003
  149.87231614]]


In [8]:
for i in range(6):
    show_img(faces[i,:])
    


Perform Incremental PCA, and compare it with the counterpart i.e. :
    - batch PCA
    - PCA trained only by the first subset
    
in terms of :
    - training time
    - reconstruction error
    - face recognition accuracy.
    
Show and discuss, including: how accurate your incremental method is, what important
parameters in the method are (and how they are set). Provide your own discussions and
measurements to support. 